In [2]:
import spacy
import re
import numpy as np
import pandas as pd

nlp = spacy.load('en')

In [2]:
nlp

In [4]:
data = pd.read_csv("kaggle.csv")
data = data.iloc[:,2:]
ex = data.Body[0]
ex = ex.replace("\n", " ")

In [5]:
ex

'Image copyright Getty Images On Sunday morning, Donald Trump went off on a Twitter tirade against a member of his own party. This, in itself, isn\'t exactly huge news. It\'s far from the first time the president has turned his rhetorical cannons on his own ranks. This time, however, his attacks were particularly biting and personal. He essentially called Tennessee Senator Bob Corker, the chair of the powerful Senate Foreign Relations Committee, a coward for not running for re-election. He said Mr Corker "begged" for the president\'s endorsement, which he refused to give. He wrongly claimed that Mr Corker\'s support of the Iranian nuclear agreement was his only political accomplishment. Unlike some of his colleagues, Mr Corker - free from having to worry about his immediate political future - didn\'t hold his tongue. Skip Twitter post by @SenBobCorker It\'s a shame the White House has become an adult day care center. Someone obviously missed their shift this morning. — Senator Bob Cork

### Tokenize, remove stop words and lemmatize

In [14]:
doc = nlp(ex)

words =  [token.text for token in doc if token.is_stop != True and token.is_punct != True]

import nltk
from nltk.stem.porter import *  

stemmer = PorterStemmer()  

counter = 0
for token in words: 
    words[counter] = stemmer.stem(token)
    counter += 1
    
res = str()
for i in words:
    res += " " + i

doc = nlp(res.strip())

temp = str()
for token in doc:
    temp += " " + token.lemma_
    
doc = nlp(temp.strip())

In [15]:
doc

london reuter flag fli comedi drama vietnam war veteran reson trump 's america despit period set actor bryan cranston say sunday screen london film festiv set unit state decemb 2003 u.s . forc iraq drag saddam hussein spider hole stori age serviceman reunit buri son kill action presid donald trump say total destroy north korea character dinner militari command calm storm cranston say flag fli time remind effect normal american ill advis militari campaign think lot relev today sen today 's clear cut far intent govern militari cranston acclaim lead role tv drama break bad tell reuter world war good war clear present danger stop mad man vietnam iraq lot question troop citizen right thing purpos cast member bryan cranston r director richard linklat pose festiv director clair stewart arriv premier flag fli british film institut bfi london film festiv odeon leicest squar london britain octob 8 2017 reuter afolabi sotund flag fli produc amazon studio direct co write richard linklat great crit

### Get PoS Tag

In [16]:
postag_arr = np.zeros(0)
for i in doc:
    postag_arr = np.append(postag_arr, i.tag_)

### Get Common Words

In [17]:
from collections import Counter

word_freq = Counter(words)
common_words = word_freq.most_common(5)
print(common_words)

[('flag', 7), ('fli', 7), ('london', 6), ('’s', 6), ('cranston', 6)]


### Get NER

In [18]:
labels = set([w.label_ for w in doc.ents]) 
for label in labels: 
    entities = [e.string for e in doc.ents if label==e.label_] 
    entities = list(set(entities)) 
    print(label,entities)

CARDINAL ['2014 ', 'half ', '8 2017 ']
DATE ['sunday ', '2003 ', 'today ']


HAVEN'T TRY BELOW!!

In [19]:
from gensim.corpora import Dictionary
from gensim.models.tfidfmodel import TfidfModel

docs_dict = Dictionary(docs)
docs_dict.filter_extremes(no_below=20, no_above=0.2)
docs_dict.compactify()

docs_corpus = [docs_dict.doc2bow(doc) for doc in docs]
model_tfidf = TfidfModel(docs_corpus, id2word=docs_dict)
docs_tfidf  = model_tfidf[docs_corpus]
docs_vecs   = np.vstack([sparse2full(c, len(docs_dict)) for c in docs_tfidf])

ModuleNotFoundError: No module named 'gensim'

In [20]:
postag_arr

array(['JJ', 'NN', 'NN', 'JJ', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN',
       'POS', 'NN', 'NN', 'NN', 'VBN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN',
       'JJ', 'NN', 'NN', 'VBD', 'NN', 'NN', 'NN', 'CD', 'NNP', '.', 'VBZ',
       'JJ', 'VBP', 'JJ', 'NN', 'NN', 'NN', 'NNS', 'NN', 'NN', 'NN', 'NN',
       'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'VBP', 'JJ', 'JJ', 'NN', 'NN',
       'NN', 'NN', 'JJ', 'NN', 'JJ', 'NN', 'NN', 'VBP', 'NN', 'JJ', 'NN',
       'NN', 'NN', 'JJ', 'JJ', 'JJ', 'JJ', 'JJ', 'NN', 'NN', 'NN', 'NNS',
       'NN', 'VBD', 'NN', 'POS', 'JJ', 'VBN', 'JJ', 'JJ', 'VBP', 'JJ',
       'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'VB', 'JJ', 'NN', 'NN', 'NN',
       'NN', 'JJ', 'NN', 'JJ', 'JJ', 'NN', 'VB', 'JJ', 'NN', 'VB', 'JJ',
       'NN', 'NN', 'NN', 'NN', 'JJ', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN',
       'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'JJ', 'NN',
       'NN', 'JJ', 'JJ', 'NN', 'NN', 'NN', 'JJ', 'NN', 'NN', 'NN', 'NN',
       'NN', 'NN', 'NN', 'VBZ', 'CD', 'CD', 

In [22]:
def NN_percent(x):
    return sum([k == "NN" for k in x])/ len(x)

In [23]:
def NN_percent(x):
    return sum([k == "NN" for k in x])

0.6561514195583596